# Automatic differentiation with `autograd`

In machine learning, we *train* models to get better and better as a function of experience.  
Usually, *getting better* means minimizing a *loss function*, i.e. a score that answers -- how *bad* is our model?  
With neural networks, we choose loss functions to be differentiable with respect to our parameters.  
Put simply, this means that for each of the model's parameters, we can determine how much *increasing* or *decreasing* it might affect the loss.  
While the calculations are straightforward, for complex models, working it out by hand can be a pain.  
_MXNet_'s `autograd` package expedites this work by automatically calculating derivatives.  
And while most other libraries require that we compile a symbolic graph to take automatic derivatives, ``mxnet.autograd``, like PyTorch, allows you to take derivatives while writing  ordinary imperative code.  
Every time you make a pass through your model, ``autograd`` builds a graph on the fly, through which it can immediately backpropagate gradients.  
Let's go through it step by step.  
For this tutorial, we'll only need to import ``mxnet.ndarray``, and ``mxnet.autograd``.

In [1]:
import mxnet as mx
from mxnet import nd, autograd
mx.random.seed(1)

## Attaching gradients